In [ ]:
import sys

rootpath = '/home/student/ROI/SparkProgram/'
datapath = f'{rootpath}datasets/'
sys.path.append(rootpath)
import pyspark_helpers as pyh
from pyspark_helpers import *
sc, spark, conf = initspark()

import pandas as pd
import matplotlib as mp
import numpy
from matplotlib import pyplot as plt


### Using the bank dataset see if you can find the natural number of clusters

In [ ]:
filename = 'bank.csv'
df = spark.read.csv(f'{datapath}/finance/{filename}', header = True, inferSchema = True)
display(df)
df.printSchema()
dfRaw = df

### Suggested features are
numeric: 'age', 'balance','duration','pdays','previous'
categorical_features: 'job','marital','education',      'housing','loan','campaign','deposit'
target_label: None

#### Hint:
use helper function pyh.MakeMLDataFrame

In [ ]:
numeric_features = ['age', 'balance','duration','pdays','previous']
categorical_features = ['job','marital','education','housing','loan','campaign','deposit']
target_label = None

dfML = pyh.MakeMLDataFrame(df, categorical_features, numeric_features, target_label, False)
display(dfML)
dfML.printSchema()


### Make an elbow plot
Hint: Use the helper function pyh.plot_elbow

In [ ]:
%matplotlib inline
pyh.plot_elbow(dfML.select('features'))

### Do a cluster analysis with the suggested number of clusters

In [ ]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
CLUSTERS = 8
kmeans = KMeans().setK(CLUSTERS).setSeed(1)
model = kmeans.fit(dfML.select('features'))
predictions = model.transform(dfML)
centroids = model.clusterCenters()
print(centroids)
display(predictions)

### See if you can predict someone's age from the dataset
suggested numeric features: 'balance','duration','pdays','previous'
categorical features: 'job','marital','education','housing',
'loan','campaign','deposit'
target: 'age'

In [ ]:
# import importlib
# importlib.reload(pyh)

numeric_features = ['balance','duration','pdays','previous']
categorical_features = ['job','marital','education','housing','loan','campaign','deposit']
target_label = 'age'

dfML = pyh.MakeMLDataFrame(dfRaw, categorical_features, numeric_features, target_label, False)
display(dfML)
dfML.printSchema()


### Split the dataset

In [ ]:
train, test = dfML.randomSplit([.7,.3], seed = 1000)
print (f'Training set row count {train.count()}')
print (f'Testing set row count {test.count()}')


### Try linear regression

In [ ]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol = 'features', labelCol='target', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lrModel = lr.fit(train)
print("Coefficients: " + str(lrModel.coefficients))
print("Intercept: " + str(lrModel.intercept))

print("Root Mean Squared Error: {}\nR Squared (R2) {}".format(lrModel.summary.rootMeanSquaredError, lrModel.summary.r2))


### See how well it did some predictions

In [ ]:
lrPredictions = lrModel.transform(test)
display(lrPredictions.select("prediction","target","features"), 30)
from pyspark.ml.evaluation import RegressionEvaluator
lrEvaluator = RegressionEvaluator(predictionCol="prediction", labelCol="target",metricName="r2")
testResult = lrModel.evaluate(test)
print("Root Mean Squared Error on Test set: {}".format(testResult.rootMeanSquaredError))
